In [3]:
from flare.data_science.features import domain_tld_extract
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import random
import json



In [5]:
data = [ json.loads(line) 
            for line in open('files/large_eve.json', 'r', encoding='utf-8') ]

EJERCICIo 3

In [6]:
print(len(data))

746909


In [8]:
data[0:2]

[{'timestamp': '2017-07-22T17:33:15.784100-0500',
  'flow_id': 283570730437312,
  'pcap_cnt': 18519,
  'event_type': 'alert',
  'vlan': 130,
  'src_ip': '192.168.203.200',
  'src_port': 2328,
  'dest_ip': '192.181.145.109',
  'dest_port': 445,
  'proto': 'TCP',
  'alert': {'action': 'allowed',
   'gid': 1,
   'signature_id': 2001569,
   'rev': 15,
   'signature': 'ET SCAN Behavioral Unusual Port 445 traffic Potential Scan or Infection',
   'category': 'Misc activity',
   'severity': 3}},
 {'timestamp': '2017-07-22T17:33:16.571273-0500',
  'flow_id': 1519048138171115,
  'pcap_cnt': 21531,
  'event_type': 'ssh',
  'vlan': 140,
  'src_ip': '192.168.204.60',
  'src_port': 62990,
  'dest_ip': '192.168.26.254',
  'dest_port': 22,
  'proto': 'TCP',
  'ssh': {'client': {'proto_version': '2.0',
    'software_version': 'OpenSSH_5.0'},
   'server': {'proto_version': '1.99', 'software_version': 'Cisco-1.25'}}}]

In [9]:
dns_data = len([x for x in data if 'dns' in x])
print(dns_data)

15749


In [10]:
dns_data = [x for x in data if 'dns' in x]
random.sample(dns_data, 2)

[{'timestamp': '2017-07-22T19:08:35.796488-0500',
  'flow_id': 1438444861728383,
  'pcap_cnt': 2558053,
  'event_type': 'dns',
  'vlan': 150,
  'src_ip': '192.168.207.4',
  'src_port': 53,
  'dest_ip': '192.168.205.188',
  'dest_port': 36542,
  'proto': 'UDP',
  'dns': {'type': 'answer',
   'id': 43797,
   'rcode': 'NXDOMAIN',
   'rrname': 'versioncheck.addons.mozilla.org'}},
 {'timestamp': '2017-07-22T19:29:19.470346-0500',
  'flow_id': 1119708977769802,
  'pcap_cnt': 3355631,
  'event_type': 'dns',
  'vlan': 150,
  'src_ip': '192.168.205.188',
  'src_port': 42994,
  'dest_ip': '192.168.207.4',
  'dest_port': 53,
  'proto': 'UDP',
  'dns': {'type': 'query',
   'id': 48722,
   'rrname': 'ftp.usf.edu',
   'rrtype': 'A',
   'tx_id': 0}}]

In [11]:
data_norm = pd.json_normalize(data)
a_records = data_norm[data_norm['dns.rrtype']==('A')][['dns.rrname','dns.rrtype']]
a_records

,dns.rrname,dns.rrtype
2,api.wunderground.com,A
38,stork79.dropbox.com,A
39,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A
43,api.wunderground.com,A
44,safebrowsing.clients.google.com.home,A
...,...,...
714913,secure.informaction.com.hsd1.pa.comcast.net,A
715127,192.168.22.201:.stayonline.net,A
715804,wpad.aol.aoltw.net,A
715910,client-software.real.com,A


In [12]:
print("Numero de registros A: ", len(a_records))
unique_records_domain = a_records['dns.rrname'].unique()
print("Numero de registros A unicos: ", len(unique_records_domain))

Numero de registros A:  2849
Numero de registros A unicos:  177


In [19]:
df = pd.DataFrame(unique_records_domain, columns=['domain'])

domain_tld = []
dnsData = df.reset_index()
for col, row in dnsData.iterrows():
    domain_tld.append(domain_tld_extract(row['domain']))

dnsData['domain_tld'] = domain_tld

dnsData = dnsData[['domain_tld']]
dnsData.head()



,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,safebrowsing.clients.google.com.home
4,mozilla.com


In [20]:
tldRemoveCaracter = ['', ' ', '192.168.22.110phpmyadmin', '192.168.22.110phpmyadmin.localdomain', '"192.168.206.56"', '192.168.26-27.0', '192.168.21.1201', '1922.168.22.254', '1922.168.22.254.home', '192.168.21-28.0.home', '192.168.22.201:', '192.168.22.201:.stayonline.net']
cleanDnsData = dnsData[~dnsData['domain_tld'].isin(tldRemoveCaracter)]

# remove duplicates
cleanDnsData = cleanDnsData.drop_duplicates(subset=['domain_tld'])

cleanDnsData



,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,safebrowsing.clients.google.com.home
4,mozilla.com
...,...
166,192.168.21-28.0
167,ejfodfmfxlkgifuf.xyz
172,ntp.ubuntu.com.localdomain
174,verisign.com


In [21]:
print("TLD unicos: ", len(cleanDnsData))

TLD unicos:  102


In [26]:
import clasificador

new_df = clasificador.clasificacion(cleanDnsData)

new_df



c:\Users\venta\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,domain_tld,isDGA
0,wunderground.com,0
1,dropbox.com,1
2,aoltw.net,1
3,safebrowsing.clients.google.com.home,0
4,mozilla.com,0
...,...,...
166,192.168.21-28.0,1
167,ejfodfmfxlkgifuf.xyz,1
172,ntp.ubuntu.com.localdomain,1
174,verisign.com,0


In [27]:
df_DGA = new_df[(new_df['isDGA'] == 1)]

print("DGA: ", len(df_DGA))



DGA:  35


In [28]:


df_umbrella = pd.read_csv("./flare/data/umbrella/top-1m.csv")

df_umbrella



,1,netflix.com
0,2,api-global.netflix.com
1,3,prod.netflix.com
2,4,push.prod.netflix.com
3,5,google.com
4,6,microsoft.com
...,...,...
999994,999996,taipei-a1.gp.schneider-electric.com
999995,999997,taiwantoday.tw
999996,999998,takethebestpics.com
999997,999999,takiedela.ru
